In [1]:
### till now case-study is of 58 mins !
### in latest release it will be of 1:10 mins !

### https://www.kaggle.com/andresionek/geospatial-analysis-of-brazilian-e-commerce

#### https://www.kaggle.com/aestheteaman01/spatial-analysis-of-zomato-restaurants

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### so that u dont have warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
import folium
folium.__version__

'0.12.1'

In [4]:
import geopy
geopy.__version__

'2.2.0'

In [5]:
## read zomato.csv
df=pd.read_csv(r'G:\Data Science projects\EDA_Projects\zomato-bangalore-restaurants/zomato.csv')
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [6]:
df.shape

(51717, 17)

### Data Pre-processing of Location feature :

In [7]:
df.isna().sum()
## df.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [8]:
### So first & foremost  we have to clean this location 
 # feature in order to come up with such results..

In [9]:
### missing values in location feature..
df['location'].isna().value_counts()

False    51696
True        21
Name: location, dtype: int64

In [10]:
### As we have few missing values in location feature ,
# then we can drop that..

In [11]:
df.dropna(subset=['location'],inplace=True)
## df.dropna(axis='index',subset=['location'],inplace=True)

In [12]:
df['location'].isna().sum()
## now this feature is ready..

0

In [13]:
len(df['location'].unique())

93

In [14]:
### Lets add 'Bangalore, India' so that u will
# get more more accurate geographical co-ordinates..

df['location']=df['location'] + \
            ', Bangalore, Karnataka, India'

In [15]:
rest_loc=pd.DataFrame()

In [16]:
rest_loc['Name']=df['location'].unique()

In [17]:
rest_loc.head(2)

,Name
0,"Banashankari, Bangalore, Karnataka, India"
1,"Basavanagudi, Bangalore, Karnataka, India"


In [18]:
rest_loc.shape

(93, 1)

In [19]:
## rest_loc=pd.DataFrame({"Name":df['location'].unique()})

#  how to extract geographical-coordinates(latitudes & longitudes )

### definition of latitudes & Longitudes :
    The globe is split into an imaginary 360 sections from both top to bottom (north to south) 
    and 180 sections from side to side (west to east). The sections running from top to bottom on a globe are called
    longitude and the sections running from side to side on a globe are called latitude.
    
    Latitude is the measurement of distance north or south of the Equator.
    
    Every location on earth has a global address. Because the address is in numbers, 
    people can communicate about location no matter what language they might speak. 
    A global address is given as two numbers called coordinates. 
    The two numbers are a location's latitude number and its longitude number ("Lat/Long").

#### https://www.udemy.com/instructor/communication/qa/16391478/detail?course=3565275

In [20]:
## now the immediate question is-how to extract lat & lon of location ??

In [21]:
### so using address or location feature  ,we can 
# extract what the geographical co-ordinates are !

### so just for simplicity first we will learn how to 
# extract it using 'location' feature ,then in the next session  ,
# we will learn how to do it using 'address' feature..

#### extract geographical-coordinates(latitudes & longitudes) using 'location' feature

In [22]:
## !pip install geopy

In [23]:
from geopy.geocoders import Nominatim

In [24]:
## set timeout=None to get rid of timeout error

geolocator=Nominatim(user_agent="app",timeout=None)

In [25]:
'''lat_lon=[]
for location in locations['Name']:
    location = geolocator.geocode(location)
    if location is None:
        lat_lon.append(np.nan)
    else:    
        geo=(location.latitude,location.longitude)
        lat_lon.append(geo)'''

### then unzip it

"lat_lon=[]\nfor location in locations['Name']:\n    location = geolocator.geocode(location)\n    if location is None:\n        lat_lon.append(np.nan)\n    else:    \n        geo=(location.latitude,location.longitude)\n        lat_lon.append(geo)"

In [26]:
rest_loc.head(2)

,Name
0,"Banashankari, Bangalore, Karnataka, India"
1,"Basavanagudi, Bangalore, Karnataka, India"


In [27]:
## takes 2 mins
lat=[]
lon=[]
for location in rest_loc['Name']:
    location = geolocator.geocode(location)    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)
        

In [28]:
rest_loc['lat']=lat
rest_loc['lon']=lon

In [29]:
rest_loc.head(3)

,Name,lat,lon
0,"Banashankari, Bangalore, Karnataka, India",12.915221,77.573598
1,"Basavanagudi, Bangalore, Karnataka, India",12.941726,77.575502
2,"Mysore Road, Bangalore, Karnataka, India",12.950636,77.537783


### if u guys are facing : GeocoderUnavailable SSL error
https://www.udemy.com/instructor/communication/qa/16390726/detail?course=3565275

### Geocode SErvice Error :
https://www.udemy.com/instructor/communication/qa/16062844/detail?course=3565275

In [30]:
### lets figure it out whether we have misssing value or not !
rest_loc.isnull().sum()

Name    0
lat     2
lon     2
dtype: int64

In [67]:
rest_loc[rest_loc['lat'].isna()]

,Name,lat,lon
45,"St. Marks Road, Bangalore, Karnataka, India",NaN,NaN
79,"Rammurthy Nagar, Bangalore, Karnataka, India",NaN,NaN


In [32]:
### Rammurthy Nagar is situated in north-eastern part of Bangalore
### lets find out whether Rammurthy Nagar has geographical 
## attributes or not..

loc=geolocator.geocode(rest_loc['Name'][73])
hasattr(loc,'latitude')

### it means that "Rammurthy Nagar" doesn'thave property as latitude , 
## ie it is able to find geo-graphical co-ordinates for "Rammurthy Nagar".


True

In [33]:

### lets take help of google in such circumstances..
### google it as "co-ordinates of Rammurthy Nagar bengalore" & find co-ordinates as :
### 13.0163° N, 77.6785° E

In [68]:
rest_loc['lat'][45]=13.0163
rest_loc['lon'][45]=77.6785

In [35]:
### for St. Marks Road ,
### 12.969332 (Lat) & 77.600509(Lon)

In [69]:
rest_loc['lat'][79]=12.969332
rest_loc['lon'][79]=77.600509

In [70]:
rest_loc.isnull().sum()

Name    0
lat     0
lon     0
dtype: int64

In [38]:
rest_loc.head(2)

,Name,lat,lon
0,"Banashankari, Bangalore, Karnataka, India",12.915221,77.573598
1,"Basavanagudi, Bangalore, Karnataka, India",12.941726,77.575502


# b..how to extract geographical-
    coordinates using 'address' feature

In [39]:
df['address']

0        942, 21st Main Road, 2nd Stage, Banashankari, ...
1        2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...
2        1112, Next to KIMS Medical College, 17th Cross...
3        1st Floor, Annakuteera, 3rd Stage, Banashankar...
4        10, 3rd Floor, Lakshmi Associates, Gandhi Baza...
                               ...                        
51712    Four Points by Sheraton Bengaluru, 43/3, White...
51713    Number 10, Garudachar Palya, Mahadevapura, Whi...
51714    Sheraton Grand Bengaluru Whitefield Hotel & Co...
51715    Sheraton Grand Bengaluru Whitefield Hotel & Co...
51716    ITPL Main Road, KIADB Export Promotion Industr...
Name: address, Length: 51696, dtype: object

In [40]:
df['address'][0]

'942, 21st Main Road, 2nd Stage, Banashankari, Bangalore'

In [41]:
## set timeout=None to get rid of timeout error

geolocator=Nominatim(user_agent="app",timeout=None)

In [42]:
loc=geolocator.geocode(df['address'][0])
hasattr(loc,'latitude')
## hasattr Return whether the loc has an attribute of latitude or not..


### it means address doesnt have any property as latitude , 
# ie it is unable to find any geo-graphical for address feature..

False

In [43]:
loc=geolocator.geocode(rest_loc['Name'][0])
hasattr(loc,'latitude')


True

In [44]:
def get_latitude(x):
    loc=geolocator.geocode(x)
    if hasattr(loc,'latitude') and \
                (loc.latitude is not None): 
        return loc.latitude
    else:
        print('Not found')

In [45]:
rest_loc['Name'][0]

'Banashankari, Bangalore, Karnataka, India'

In [46]:
get_latitude(rest_loc['Name'][0])

12.9152208

In [47]:
get_latitude(df['address'][0])

Not found


## now how to find geo-graphical co-ordinates using address
    For this u have to use structured query  , ie we have to 
        tell Nominatim that this is Street , this is city , 
        this is country , this is postalcode..

In [48]:
df['address'][0]

'942, 21st Main Road, 2nd Stage, Banashankari, Bangalore'

In [49]:
address={'street':'21st Main Road','city':'Bangalore',\
         'country':'India','state':'Karnataka'}

In [50]:
get_latitude(address)

13.0041454

#### have a look
## https://www.py4u.net/discuss/195256

## https://nominatim.org/release-docs/develop/api/Search/

# 1..  Where are most number of restaurants located in Bengalore city ?

In [51]:
### in order to understand what are those areas where 
# we have most number of restaurants ?

In [71]:
Rest_locations=pd.DataFrame(df['location'].\
                    value_counts().reset_index())
Rest_locations.columns=['Name','count']
Rest_locations.head()

,Name,count
0,"BTM, Bangalore, Karnataka, India",5124
1,"HSR, Bangalore, Karnataka, India",2523
2,"Koramangala 5th Block, Bangalore, Karnataka, I...",2504
3,"JP Nagar, Bangalore, Karnataka, India",2235
4,"Whitefield, Bangalore, Karnataka, India",2144


In [53]:
### Now we can say that These are my top locations 
#  where most of my restaurants are located..

In [54]:
#### lets create Heatmap of this results so that it becomes more 
## user-friendly..now In order to perform Spatial Anlysis
# (Geographical Analysis) , we need latitudes & longitudes of every location..

In [55]:
### so lets merge both the dataframes or append one 
# into another in order to get geographical co-ordinates..

In [72]:
Restaurant_locations=Rest_locations.merge(rest_loc,\
                                  on='Name',how="left")
Restaurant_locations.head()

,Name,count,lat,lon
0,"BTM, Bangalore, Karnataka, India",5124,12.911276,77.604565
1,"HSR, Bangalore, Karnataka, India",2523,12.911623,77.638862
2,"Koramangala 5th Block, Bangalore, Karnataka, I...",2504,12.923525,77.622287
3,"JP Nagar, Bangalore, Karnataka, India",2235,12.907251,77.578271
4,"Whitefield, Bangalore, Karnataka, India",2144,12.969637,77.749745


In [57]:
## Restaurant_locations.to_csv('E:\Spatial Analysis\Zomato/zomato_locations.csv',index=False)

#### now in order to show-case it via Map(Heatmap) 
    ,first we need to create BaseMap so that I can map 
     our Heatmap on top of BaseMap !

### https://www.udemy.com/instructor/communication/qa/16581888/detail?course=3565275

In [58]:
### Geographic heat maps are used to identify where 
# something occurs, and demonstrate areas of high and low density...

In [59]:
##!pip install folium

In [60]:
import folium

In [61]:
### search as 'co-ordinates of bengalore' in 
# order to get 12.9716° N, 77.5946° E

def generateBaseMap(default_location=[12.97, 77.59],\
                    default_zoom_start=12):
    
    base_map = folium.Map(location=default_location, 
                          zoom_start=default_zoom_start)
    return base_map

In [73]:
basemap=generateBaseMap()

In [75]:
from folium.plugins import HeatMap

In [76]:
HeatMap(Restaurant_locations[['lat','lon','count']]\
        .values.tolist(),zoom=20,radius=15).add_to(basemap)

In [77]:
basemap

### Conclusions : in the city centre area ,majority of the 
                    Restaurants are avaiable :

### how to add Colorbar to our Heatmap :
https://www.udemy.com/instructor/communication/qa/16288518/detail?course=3565275

### Maximum Recursion Depth Exceeded
https://www.udemy.com/course/spatial-data-science-in-python/learn/lecture/23309584#questions/15042140/

In [79]:
## increase your recursion limit , if u will get a error as Maximum Recursion Depth Exceeded
## use below code :

In [80]:
import sys

In [81]:
sys.getrecursionlimit()

3000

In [82]:
sys.setrecursionlimit(10000)

In [83]:
sys.getrecursionlimit()

10000

##### Geo Analysis: where are the restaurants located in Bengaluru using Marker Cluster?

In [84]:
from folium.plugins import FastMarkerCluster

In [85]:
basemap=generateBaseMap()

In [86]:
# Plugin: FastMarkerCluster
FastMarkerCluster(data=Restaurant_locations[['lat','lon','count']]\
                  .values.tolist()).add_to(basemap)

basemap

In [87]:
#Importing the Libraries
#Mention the latitudes and Longitudes here
latitude = '12.9703944526'
longitude = '77.6447132975'
import folium
import matplotlib.pyplot as plt
%matplotlib inline 

#Creating the Map instance with latitudes and longitudes 
#   set to IndiraNagar
indiranagar_map = folium.Map(location=[latitude,longitude],\
                             zoom_start=16)

#Plotting all the restaurants near Indiranagar's Latitude within 500 meters

#Plotting the indiranagar coordinates
a = folium.map.FeatureGroup()
a.add_child(folium.CircleMarker([latitude,longitude],radius=10,color='black',fill_color='black',popup="IndiraNagar"))
indiranagar_map.add_child(a)

#Checking the plot
indiranagar_map

# 2.. where are the restaurants situated with high average rate?

In [88]:
df.head(2)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,"Banashankari, Bangalore, Karnataka, India",Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,"Banashankari, Bangalore, Karnataka, India",Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari


In [89]:
len(df['location'].unique())

93

In [90]:
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', nan, '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [91]:
### now it needs cleaning..

In [92]:
df['rate'].isnull().sum()

7754

In [93]:
df.shape

(51696, 17)

In [94]:
### ie approximately 15% of your data belongs to missing values..

In [95]:
df.dropna(subset=['rate'],inplace=True)
##df.dropna(axis=0,subset=['rate'],inplace=True)

In [96]:
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [97]:
df.replace('NEW','0',inplace=True)
df.replace('-','0',inplace=True)



## Note:the decision of replacing {New->'0'} & (->'0') can be changed 
       ,as it depends upon domain expertise & your business analyst..
## u have to maintain data-type while doing this ,
   thats why u have to use str like '0' not a int 0..

## else u will end up getting error in the upcoming process..

#### now we need rating as 4.1  ,so there are couple of ways to do it (4 ways) :
    1.. using simple function
    2.. using lambda
    3.. using loops
    4.. using map function

In [98]:
'4.1/5'.split('/')[0]

'4.1'

In [99]:
float('4.1/5'.split('/')[0])

4.1

In [100]:
def split(x):
    return float(x.split('/')[0])

In [101]:
df['rate'][0].split('/')[0]

'4.1'

In [102]:
df['rating']=df['rate'].apply(split)
## df['rate'].apply(lambda x : x.split('/')[0])

In [103]:
df['rating'].dtype

dtype('float64')

In [104]:
### or df['rating']=pd.to_numeric(df['rating'])
### df['rating']=df['rating'].astype(float)

In [105]:
### needs to work on it :
### giving error as key-error:'rate':df.apply(lambda row:float(row['rate'].split('/')[0]))

In [106]:
'''
ratings=[]
for rate in df['rate']:
    ratings.append(float(rate.split('/')[0]))


'''

"\nratings=[]\nfor rate in df['rate']:\n    ratings.append(float(rate.split('/')[0]))\n\n\n"

In [107]:
### list(map(split,df['rate']))

In [108]:
df['rating'].unique()

array([4.1, 3.8, 3.7, 3.6, 4.6, 4. , 4.2, 3.9, 3.1, 3. , 3.2, 3.3, 2.8,
       4.4, 4.3, 0. , 2.9, 3.5, 2.6, 3.4, 4.5, 2.5, 2.7, 4.7, 2.4, 2.2,
       2.3, 4.8, 4.9, 2.1, 2. , 1.8])

In [109]:
df.head(2)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,"Banashankari, Bangalore, Karnataka, India",Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,"Banashankari, Bangalore, Karnataka, India",Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.1


In [110]:
df.groupby(['location'])['rating'].mean()

location
BTM, Bangalore, Karnataka, India                  3.296128
Banashankari, Bangalore, Karnataka, India         3.373292
Banaswadi, Bangalore, Karnataka, India            3.362926
Bannerghatta Road, Bangalore, Karnataka, India    3.271677
Basavanagudi, Bangalore, Karnataka, India         3.478185
                                                    ...   
West Bangalore, Bangalore, Karnataka, India       2.020000
Whitefield, Bangalore, Karnataka, India           3.384170
Wilson Garden, Bangalore, Karnataka, India        3.257635
Yelahanka, Bangalore, Karnataka, India            3.640000
Yeshwantpur, Bangalore, Karnataka, India          3.502679
Name: rating, Length: 92, dtype: float64

In [111]:
df.groupby(['location'])['rating'].mean().\
            sort_values(ascending=False)

location
Lavelle Road, Bangalore, Karnataka, India             4.042886
St. Marks Road, Bangalore, Karnataka, India           4.017201
Koramangala 3rd Block, Bangalore, Karnataka, India    3.978756
Sankey Road, Bangalore, Karnataka, India              3.965385
Church Street, Bangalore, Karnataka, India            3.963091
                                                        ...   
Electronic City, Bangalore, Karnataka, India          3.041909
Bommanahalli, Bangalore, Karnataka, India             2.926752
Hebbal, Bangalore, Karnataka, India                   2.880000
North Bangalore, Bangalore, Karnataka, India          2.385714
West Bangalore, Bangalore, Karnataka, India           2.020000
Name: rating, Length: 92, dtype: float64

In [112]:
### Lets insert this Avg_rating into our'Restaurant_locations'dataframe..

In [113]:
ratings=df.groupby(['location'])['rating'].mean().\
            sort_values(ascending=False).reset_index()

### df.groupby(['location'],as_index=False)['rating'].mean().\
##           sort_values(by='rating',ascending=False)

In [114]:
ratings.columns=['Name','avg_rating']
ratings

,Name,avg_rating
0,"Lavelle Road, Bangalore, Karnataka, India",4.042886
1,"St. Marks Road, Bangalore, Karnataka, India",4.017201
2,"Koramangala 3rd Block, Bangalore, Karnataka, I...",3.978756
3,"Sankey Road, Bangalore, Karnataka, India",3.965385
4,"Church Street, Bangalore, Karnataka, India",3.963091
...,...,...
87,"Electronic City, Bangalore, Karnataka, India",3.041909
88,"Bommanahalli, Bangalore, Karnataka, India",2.926752
89,"Hebbal, Bangalore, Karnataka, India",2.880000
90,"North Bangalore, Bangalore, Karnataka, India",2.385714


In [115]:
Ratings_locations=ratings.merge(rest_loc,\
                            on='Name',how="left")
Ratings_locations.head()

,Name,avg_rating,lat,lon
0,"Lavelle Road, Bangalore, Karnataka, India",4.042886,12.968305,77.596488
1,"St. Marks Road, Bangalore, Karnataka, India",4.017201,13.016300,77.678500
2,"Koramangala 3rd Block, Bangalore, Karnataka, I...",3.978756,12.927187,77.626625
3,"Sankey Road, Bangalore, Karnataka, India",3.965385,13.003050,77.584044
4,"Church Street, Bangalore, Karnataka, India",3.963091,12.969332,77.600509


In [116]:
Ratings_locations.isna().sum()

Name          0
avg_rating    0
lat           0
lon           0
dtype: int64

In [117]:
basemap=generateBaseMap()

In [118]:
HeatMap(Ratings_locations[['lat','lon','avg_rating']],\
                zoom=20,radius=15).add_to(basemap)
basemap

#### Inference : Above are the restaurants with high average rate

In [119]:
### Another way of doing it :

In [120]:
'''
locations=df.groupby(['location'])['rating'].mean().index
avg_rating=df.groupby(['location'])['rating'].mean().values


geolocator=Nominatim(user_agent="app")
lat=[]
lon=[]
for location in locations:
    location = geolocator.geocode(location)    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)
        
        
rating=pd.DataFrame()
rating['location']=locations
rating['lat']=lat
rating['lon']=lon
rating['avg_rating']=avg_rating

rating=rating.dropna()

HeatMap(rating[['lat','lon','avg_rating']],zoom=20,radius=15).add_to(basemap)
basemap
'''

'\nlocations=df.groupby([\'location\'])[\'rating\'].mean().index\navg_rating=df.groupby([\'location\'])[\'rating\'].mean().values\n\n\ngeolocator=Nominatim(user_agent="app")\nlat=[]\nlon=[]\nfor location in locations:\n    location = geolocator.geocode(location)    \n    if location is None:\n        lat.append(np.nan)\n        lon.append(np.nan)\n    else:\n        lat.append(location.latitude)\n        lon.append(location.longitude)\n        \n        \nrating=pd.DataFrame()\nrating[\'location\']=locations\nrating[\'lat\']=lat\nrating[\'lon\']=lon\nrating[\'avg_rating\']=avg_rating\n\nrating=rating.dropna()\n\nHeatMap(rating[[\'lat\',\'lon\',\'avg_rating\']],zoom=20,radius=15).add_to(basemap)\nbasemap\n'

### Above are the restaurants with high average rate

# 3.. In which area of Bengalore most of the North Indian Restaurants available ?

In [123]:
df.head(2)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,"Banashankari, Bangalore, Karnataka, India",Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,"Banashankari, Bangalore, Karnataka, India",Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.1


### how to extract North Indian restaurants from this data :
    Ways to do it :
    1) filter approach
    2) Pandas - pd.query()
    3) string function (str.contains() )
    4) own code : using if-else
    5) use regular regular expressions ,if u want more accurate results
    
    but just for simplicity , lets talk about filter & str.contains() first..

In [124]:
df2= df[df['cuisines']=='North Indian']
df2.head(1)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,No,3.8/5,286,+91 9980040002\r\n+91 9980063005,"Basavanagudi, Bangalore, Karnataka, India",Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[],Buffet,Banashankari,3.8


In [125]:
df2.shape

(2294, 18)

In [126]:
df2.index

Int64Index([    5,    50,    72,    87,    94,   109,   111,   131,   191,
              206,
            ...
            51549, 51552, 51585, 51592, 51595, 51597, 51600, 51612, 51622,
            51631],
           dtype='int64', length=2294)

In [127]:
df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            51704, 51705, 51706, 51707, 51708, 51709, 51711, 51712, 51715,
            51716],
           dtype='int64', length=43942)

In [128]:
### But the problem is that  : filter is good if it have only 
#    single entry in cuisines & that should be 'North Indian'

### but if cuisines is let say "North Indian Chinese" ,
#   this filter will not be handy !

In [129]:
### lets use str.contains()

In [130]:
df['cuisines'].str.contains('North Indian')

0         True
1         True
2        False
3         True
4         True
         ...  
51709     True
51711     True
51712    False
51715    False
51716     True
Name: cuisines, Length: 43942, dtype: object

In [131]:
df['cuisines'].str.contains('North Indian')

0         True
1         True
2        False
3         True
4         True
         ...  
51709     True
51711     True
51712    False
51715    False
51716     True
Name: cuisines, Length: 43942, dtype: object

In [132]:
## we will say just create a array of 0's & 1's ,where-evr it is 0 ,
#  it means at that row-index North Indian Cuisine is not available.

np.where(df['cuisines'].str.contains('North Indian')==True ,1 ,0)

array([1, 1, 0, ..., 0, 0, 1])

In [133]:
len(np.where(df['cuisines'].str.contains('North Indian')==True ,1 ,0))

43942

In [134]:
df['availablity_north_indian']=np.where(df['cuisines'].\
                str.contains('North Indian')==True ,1 ,0)

In [135]:
north_indian=df[df['availablity_north_indian']==1]

In [136]:
north_indian.head(1)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating,availablity_north_indian
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,"Banashankari, Bangalore, Karnataka, India",Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1,1


In [137]:
north_indian['location'].value_counts()

BTM, Bangalore, Karnataka, India                  2138
HSR, Bangalore, Karnataka, India                   949
Marathahalli, Bangalore, Karnataka, India          883
Whitefield, Bangalore, Karnataka, India            860
JP Nagar, Bangalore, Karnataka, India              824
                                                  ... 
Hebbal, Bangalore, Karnataka, India                  4
Central Bangalore, Bangalore, Karnataka, India       2
West Bangalore, Bangalore, Karnataka, India          1
Nagarbhavi, Bangalore, Karnataka, India              1
Peenya, Bangalore, Karnataka, India                  1
Name: location, Length: 89, dtype: int64

In [138]:
### Imagine there is no such value_counts() or imagine 
#    Interviewer will ask u - can u do it using some other ways
### bcz value_counts() is very_very straight-forward..
### u will say ok I can do it using groupby or using my own logic..

In [139]:
north_indian.groupby(['location']).size()  ## returns series

## north_indian.groupby(['location'])['rate'].size()      ## returns series
## north_indian.groupby(['location'])['rate'].count()     ## returns dataframe
## north_indian.groupby(['location']).agg({'rate':'count'})   ## returns dataframe


## north_indian.groupby(['location']).apply(lambda x:len(x))


##  def count_rows(rows):
##      return len(rows)
##  north_indian.groupby(['location']).apply(count_rows)

location
BTM, Bangalore, Karnataka, India                  2138
Banashankari, Bangalore, Karnataka, India          287
Banaswadi, Bangalore, Karnataka, India             184
Bannerghatta Road, Bangalore, Karnataka, India     618
Basavanagudi, Bangalore, Karnataka, India          168
                                                  ... 
Vijay Nagar, Bangalore, Karnataka, India            32
West Bangalore, Bangalore, Karnataka, India          1
Whitefield, Bangalore, Karnataka, India            860
Wilson Garden, Bangalore, Karnataka, India         121
Yeshwantpur, Bangalore, Karnataka, India            66
Length: 89, dtype: int64

In [140]:
cuisines=north_indian.groupby(['location']).size().\
        reset_index().rename(columns=\
                         {'location':'Name',0:'count'})

In [141]:
cuisines.head(2)

,Name,count
0,"BTM, Bangalore, Karnataka, India",2138
1,"Banashankari, Bangalore, Karnataka, India",287


In [142]:
### Now I need 'lat' & 'lon' in order to plot maps..
### so lets merge this 'cuisines' dataframe with 'rest_loc'

In [143]:
cuisines2=cuisines.merge(rest_loc,on="Name",how='left')

In [144]:
cuisines2

,Name,count,lat,lon
0,"BTM, Bangalore, Karnataka, India",2138,12.911276,77.604565
1,"Banashankari, Bangalore, Karnataka, India",287,12.915221,77.573598
2,"Banaswadi, Bangalore, Karnataka, India",184,13.014162,77.651854
3,"Bannerghatta Road, Bangalore, Karnataka, India",618,12.905971,77.601737
4,"Basavanagudi, Bangalore, Karnataka, India",168,12.941726,77.575502
...,...,...,...,...
84,"Vijay Nagar, Bangalore, Karnataka, India",32,12.965954,77.612533
85,"West Bangalore, Bangalore, Karnataka, India",1,13.022235,77.567183
86,"Whitefield, Bangalore, Karnataka, India",860,12.969637,77.749745
87,"Wilson Garden, Bangalore, Karnataka, India",121,12.948934,77.596827


In [145]:
basemap=generateBaseMap()
HeatMap(cuisines2[['lat','lon','count']].values.\
        tolist(),zoom=20,radius=15).add_to(basemap)
basemap

## Similar Problem Statements :
    In which area of Bengalore most of the South Indian Restaurants available ?
    In which area of Bengalore most of the Chinese Restaurants avaialble ?
    In which area of Bengalore most of the Mughlai Restaurants avaialble ?

In [146]:
### For above problems - Lets Automate our Analysis :
### So whenever automation comes into picture  , lets design a function :    

# 4.. Automate Above Stuffs !

In [147]:
data=df.copy()

In [148]:
def Heatmap_of_cuisine(cuisine):
    data['availablity_cuisine']=np.where(data['cuisines']\
                     .str.contains(cuisine)==True ,1 ,0)
    
    data2=data[data['availablity_cuisine']==1]
    
    cuisines=data2.groupby(['location']).size().\
            reset_index().rename(columns=\
                     {'location':'Name',0:'count'})
    
    cuisines2=cuisines.merge(rest_loc,on="Name",how='left')

    basemap=generateBaseMap()
    HeatMap(cuisines2[['lat','lon','count']].values.\
            tolist(),zoom=20,radius=15).add_to(basemap)
    return basemap

In [149]:
Heatmap_of_cuisine('Mughlai')

In [150]:
df['cuisines'].unique()

array(['North Indian, Mughlai, Chinese', 'Chinese, North Indian, Thai',
       'Cafe, Mexican, Italian', ..., 'Tibetan, Nepalese',
       'North Indian, Street Food, Biryani',
       'North Indian, Chinese, Arabian, Momos'], dtype=object)

In [153]:
Heatmap_of_cuisine('Momos')